In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import csv
import time
import librosa

import warnings
import pandas as pd
from models import *
from utils import *
import torch
import torchaudio
import numpy as np
import torch.optim as optim

warnings.filterwarnings('ignore')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
%%time 

limit = 80#None
params = {'batch_size': 20,
          'shuffle': True,
          'num_workers': 10,
          'drop_last': True}


tokenizer = tokenizer_librispeech(limit=800)
training_set = LibriSpeechDataset(limit=limit, tokenizer=tokenizer, n_frames=600)
my_dataloader = data.DataLoader(training_set, **params)

steps_per_epoch = len(training_set) // params['batch_size']

vocab_tar_size = len(tokenizer.word_index)
embedding_dim_target = 100

 ===== They are 800 transcriptions in the dataset. ===== 
CPU times: user 578 ms, sys: 89.6 ms, total: 667 ms
Wall time: 931 ms


In [ ]:
%%time
for i , data in enumerate(my_dataloader):
    example_input_batch = data[0]
    if i == 1:
        break

In [ ]:
# example_input_batch

In [ ]:
example_input_batch = example_input_batch.to(device)
encoder = EncoderCONV2DRNN(batch_size=params['batch_size'], device=device, hidden_size=512)
encoder.to(device)
# sample input
#sample_hidden = encoder.initialize_hidden_state()
#print(sample_hidden.shape)
#sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
#print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
#print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

In [ ]:
sample_output, sample_hidden = encoder(example_input_batch)

In [ ]:
# torch.randint(1, 20, (batch_size, 1)).shape

In [ ]:
#sample_hidden = encoder.last_state
decoder = Decoder(vocab_tar_size, embedding_dim_target, 512, params['batch_size'], 512).to(device)

sample_decoder_output, _, _ = decoderBASIC(torch.randint(1, 20, (params['batch_size'], 1), device=device),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

In [4]:
#encoder = EncoderRNN(batch_size=batch_size, device=device, hidden_size=512).to(device)
encoder = EncoderCONV2DRNN(batch_size=params['batch_size'], device=device, hidden_size=256).to(device)
decoder = DecoderBASIC(vocab_tar_size, embedding_dim_target, 256, params['batch_size'], 256).to(device)

encoder_optimizer = optim.Adam(encoder.parameters())
decoder_optimizer = optim.Adam(decoder.parameters())

criterion = nn.NLLLoss()

In [6]:
# Train the model
global_trainer(2, my_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer,
                                    criterion, device, params['batch_size'], tokenizer )

Epoch 2: 100%|████████████████████| 4/4 [00:41<00:00, 10.26s/it, Loss 5.5984]

Time taken for the training 0.022807 hours

